In [ ]:
!pip install geemap &> install.log

In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.3 MB/s eta 0:00:00


In [3]:
import ee
import json
import datetime
import pandas as pd
#import geemap
import os
import math
import geopandas as gpd
import re
import numpy as np

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project='potential-evap')

In [4]:
def longest_numeric_substring(string):
    return max(re.findall(r'\d+', string), key=len)

In [11]:
ET_ts_path = 'ET_TS/'
ET_ts_exported = []
for file in os.listdir(ET_ts_path):
  file_path = os.path.join(ET_ts_path,file)
  ET_ts_exported.append(file_path[-11:-4])

print(len(ET_ts_exported))

10


In [12]:
shapefiles_list = []
path = "../Shapefiles/GAGES_smooth_shp"
count_tot_shps = 0
count_shps_not_exp = 0

for file in os.listdir(path):
  if file.endswith('.shp'):
    count_tot_shps += 1
    file_path = os.path.join(path,file)
    basin_no = file_path[-11:-4] #string
    if (basin_no not in ET_ts_exported):
      count_shps_not_exp += 1
      shapefiles_list.append(file_path)

print(len(shapefiles_list))

9312


In [9]:
bounds = None
def region_et(image):
    # reduction function
    et = image.reduceRegion( #zonal statistics
        reducer = ee.Reducer.mean(),
        geometry = bounds,
        scale = 1000 # 1 km
    )
    # set the result as a metadata property in the image
    return image.set(et)

In [10]:
for i in range(len(shapefiles_list)):
# for i in range(1):
  shapefile = gpd.read_file(shapefiles_list[i])

  features = []
  for j in range(shapefile.shape[0]):
    geom = shapefile.iloc[j:j+1,:]
    jsonDict = eval(geom.to_json())
    geojsonDict = jsonDict['features'][0]
    features.append(ee.Feature(geojsonDict))

  fc = ee.FeatureCollection(features)
  bounds = fc.geometry().bounds()

  mod16 = ee.ImageCollection("MODIS/061/MOD16A2GF").select("ET").filterBounds(bounds)
  # apply the function and filter for images that were not all masked
  ET_region = mod16.map(region_et).filter(ee.Filter.neq("ET",None))

  timeseries = ET_region.aggregate_array("ET").getInfo()
  timestamp = ET_region.aggregate_array("system:time_start").getInfo()
  dates = pd.to_datetime(np.array(timestamp)*1e6)
  region_series = pd.Series(timeseries,index=dates,name="Region ET [kg/m^2/8day]")

  #-----------------------
  region_series_df = pd.DataFrame(region_series)

  # df.plot(figsize=(15,11))
  file_PathName = 'ET_TS/' + 'basin_' + shapefiles_list[i][-12:-4] + '.csv'
  print(i, '.) ', shapefiles_list[i][-11:-4])
  region_series_df.to_csv(path_or_buf=file_PathName,index=True)

0 .)  5286000
1 .)  1040000
2 .)  8086050
3 .)  7195865
4 .)  4253500
5 .)  4309500
6 .)  1044500
7 .)  4208504
8 .)  4257000
9 .)  6342260


KeyboardInterrupt: 